In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
import pandas as pd


model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("json", data_files="/home/snt/projects_lujun/mt_luxembourgish/data/training_dataset/dataset_GPT_split.jsonl")
training_dataset = dataset['train'].filter(lambda x: x['split'] == 'train').select(range(100))
validation_dataset = dataset['train'].filter(lambda x: x['split'] == 'val').select(range(10))

/home/snt/miniconda3/envs/mt_lux_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def preprocess_function(example):
    prompt = f"Translate the Luxembourgish input text into English.".upper()
    if example["input"]:
        prompt += f"\n{example['input']}"
    labels = example["translated_text"]
    tokenized = tokenizer(prompt, truncation=True, max_length=512, padding="max_length")
    with tokenizer.as_target_tokenizer():
        tokenized_labels = tokenizer(labels, truncation=True, max_length=512, padding="max_length")
    tokenized["labels"] = tokenized_labels["input_ids"]
    return tokenized

tokenized_training_dataset = training_dataset.map(preprocess_function)
tokenized_validation_dataset = validation_dataset.map(preprocess_function)


training_args = TrainingArguments(
    output_dir="./instruction_finetuned_model",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,
)

trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=tokenized_training_dataset,
    eval_dataset=tokenized_validation_dataset,
    tokenizer=tokenizer,
)

trainer.train()

model.save_pretrained("./instruction_finetuned_model")
tokenizer.save_pretrained("./instruction_finetuned_model")

/home/snt/miniconda3/envs/mt_lux_env/lib/python3.9/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_2805392/3165783323.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.805600,0.945733


KeyboardInterrupt: 